# Evaluate 930 timestamps and interchanges

### Generation: 

Check lagged correlation (-11 to +12 hours) between 930 fossil generation and CEMS fossil generation for each BA over different time bounds: 
* 2019, 2020, 2021
* Daylight savings vs. non- daylight savings

Run for both raw, shifted, and shifted + basic and rolling filtered 930 data. When shifts are correct, the best correlation in the shifted data should be at lag=0. The best correlation in the non-shifted data can indicate what shift might be appropriate. Manual inspection is required to actually decide whether and how much to lag by. 

We run with the rolling-filtered data because in some cases large errors can cause anomalous best correlations in the shifted but not filtered 930 data. 

Note: for correct timestamps, demand data in non-shifted data will be best correlated at lag=-1 because 930 uses end-of-hour timestamps while CEMS uses start-of-hour. 

### Interchange: 

Check lagged correlations between pairs of BAs with shared interchange. If timestamps are consistent, the best correlation should be at lag=0. 

We also check the sign of the best correlation between paired BAs: if they're not negatively correlated, one of the signs may be incorrect. 

### Edge cases: 

In some BAs, the shifted data still shows a best correlation at lag != 0, but inspection of the data doesn't show an obvious fix. In these cases, we do nothing and rely on `gridemissions` to make the data consistent. 




In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import timedelta

import plotly.express as px
import plotly.graph_objects as go

In [ ]:
%reload_ext autoreload
%autoreload 2

# # Tell python where to look for modules.
import sys
sys.path.append('../../../open-grid-emissions/src/')

import download_data
import load_data
from column_checks import get_dtypes
from filepaths import *
import impute_hourly_profiles
import data_cleaning
import output_data
import emissions
import validation
import gross_to_net_generation
import eia930

In [ ]:
# Data before and after shifts 
# Note: this is very slow! (~30min) because it's pivoting large files. 
lraw = []
lshift = []

for year in [2019, 2020, 2021]:
    print(year)

    r = eia930.convert_balance_file_to_gridemissions_format(year)

    s = eia930.manual_930_adjust(r)
    s = eia930.reformat_chalendar(s)
    r = eia930.reformat_chalendar(r)

    s = s[s.fuel.isin(["COL","NG","OIL"])]
    s = s.rename(columns={"UTC Time at End of Hour": "datetime_utc"})
    s = s.groupby(["datetime_utc","BA"]).sum()["generation"].reset_index()
    s = s[s.datetime_utc.dt.year == year] # filter for year

    # Filter for fossil fuels, sum by BA
    r = r[r.fuel.isin(["COL","NG","OIL"])]
    r = r.rename(columns={"UTC Time at End of Hour": "datetime_utc"})
    r = r.groupby(["datetime_utc","BA"]).sum()["generation"].reset_index()
    r = r[r.datetime_utc.dt.year == year] # filter for year
    lraw.append(r)
    lshift.append(s)

In [ ]:
raw = pd.concat(lraw, axis=0)
shifted = pd.concat(lshift, axis=0)

In [ ]:
# Load data after shifting and rolling filter 

all_rolled = []
for y in [2019, 2020, 2021]:
    rolled_930 = pd.read_csv(f"../../data/outputs/{y}/eia930/eia930_rolling.csv", index_col=0, parse_dates=True)
    rolled_930 = rolled_930[rolled_930.index.year == y]
    all_rolled.append(rolled_930)
rolled_930 = eia930.reformat_chalendar(pd.concat(all_rolled))


In [ ]:
##### Remove renewables before summing 930 

rolled_930 = rolled_930[rolled_930.fuel.isin(["COL","NG","OIL"])].groupby(["datetime_utc","BA"]).sum().reset_index()

In [ ]:
# Load files
# Aggregate by BA during loading to cut down on space
cems = pd.DataFrame()
for y in [2019, 2020, 2021]: 
    print(f"loading {y}")
    file = f"{data_folder()}/outputs/{y}/cems_cleaned_{y}.csv"
    plant_meta = pd.read_csv(f"../../data/outputs/{y}/plant_static_attributes_{y}.csv")
    c = pd.read_csv(file, index_col=0, parse_dates=['datetime_utc'])
    c = c.rename(columns={"datetime_utc":"datetime_utc"})
    c = c.merge(plant_meta[['plant_id_eia', 'plant_primary_fuel', 'ba_code']], how='left', left_index=True, right_on='plant_id_eia')
    # exclude solar power for CEMS, since we're just going to look at COL + OIL + NG in the 930 data
    c = c[c["plant_primary_fuel"] != "SUN"]
    print("Aggregating")
    if y == 2021: 
        c = c.rename(columns={"gross_generation_mwh":"net_generation_mwh"})
    cems_aggregated = c.groupby(["datetime_utc","ba_code"]).sum()["net_generation_mwh"].reset_index()
    cems = pd.concat([cems, cems_aggregated])

cems.head()

In [ ]:
plant_attributes = pd.read_csv(outputs_folder(f"{year}/plant_static_attributes_{year}.csv"), dtype=get_dtypes())

# Correlation 

In [ ]:
bas = set(raw.BA.unique())
bas.intersection_update(set(cems.ba_code.unique()))

In [ ]:
print(f"shared BAs: {len(bas)} out of {len(raw.BA.unique())} 930 BAs and {len(cems.ba_code.unique())} CEMS BAs.")

missing_cems = set(raw.BA.unique()).difference(set(cems.ba_code.unique()))
missing_930 = set(cems.ba_code.unique()).difference(set(raw.BA.unique()))
print(f"930 BAs missing in CEMS: {missing_cems}")
print(f"CEMS missing 930: {missing_930}")

In [ ]:
def find_best_cor(cems, df_eia930):
    cems = cems.pivot(columns="ba_code", index="datetime_utc", values="net_generation_mwh")
    df_eia930 = df_eia930.pivot(columns="BA", index="datetime_utc", values="generation")

    bas = set(cems.columns).intersection(set(df_eia930.columns))

    correlations = pd.DataFrame(index=bas, columns=range(-12,12), dtype=float)

    for ba in correlations.index:
        for lag in correlations.columns:
            # prepare 930: select BA 
            #eia = df_eia930[df_eia930.BA==ba]["generation"]
            # prepare CEMS: select BA
            #c = cems[cems.ba_code==ba]["net_generation_mwh"]
            # calculate 
            correlations.loc[ba,lag] = cems[ba]\
                .corr(df_eia930[ba].shift(lag))

    best = correlations.apply(lambda s: s.index[s.argmax()], axis=1).rename("best")

    correlations = pd.concat([best, correlations], axis='columns')
    return correlations

In [ ]:
cems.drop_duplicates(subset=["datetime_utc","ba_code"], inplace=True)
#rolled_930.drop_duplicates(subset=["datetime_utc","BA"], inplace=True)

In [ ]:
# Calculate best correlations for shifted (no EBA cleaning) data

cems_930_cors = pd.concat([find_best_cor(cems, shifted).best.rename("all_years"),\
    find_best_cor(cems[cems.datetime_utc.dt.year==2019],shifted[shifted.datetime_utc.dt.year==2019]).best.rename("2019"),
    find_best_cor(cems[cems.datetime_utc.dt.year==2020],shifted[shifted.datetime_utc.dt.year==2020]).best.rename("2020"),
    find_best_cor(cems[cems.datetime_utc.dt.year==2021],shifted[shifted.datetime_utc.dt.year==2021]).best.rename("2021"),
    find_best_cor(cems[(cems.datetime_utc.dt.month>=4)&(cems.datetime_utc.dt.month<=9)],
        shifted[(shifted.datetime_utc.dt.month>=4)&(shifted.datetime_utc.dt.month<=9)]).best.rename("daylight time"),
    find_best_cor(cems[(cems.datetime_utc.dt.month>=11)|(cems.datetime_utc.dt.month<=2)],
        shifted[(shifted.datetime_utc.dt.month>=11)|(shifted.datetime_utc.dt.month<=2)]).best.rename("standard time")],
    axis='columns')

cems_930_cors.to_csv("../../data/outputs/2021/cems_SHIFTEDeia930_cor_lags.csv")
#cems_930_cors

In [ ]:
# Calculate best correlations for raw data

cems_930_cors = pd.concat([find_best_cor(cems, raw).best.rename("all_years"),\
    find_best_cor(cems[cems.datetime_utc.dt.year==2019],raw[raw.datetime_utc.dt.year==2019]).best.rename("2019"),
    find_best_cor(cems[cems.datetime_utc.dt.year==2020],raw[raw.datetime_utc.dt.year==2020]).best.rename("2020"),
    find_best_cor(cems[cems.datetime_utc.dt.year==2021],raw[raw.datetime_utc.dt.year==2021]).best.rename("2021"),
    find_best_cor(cems[(cems.datetime_utc.dt.month>=4)&(cems.datetime_utc.dt.month<=9)],
        raw[(raw.datetime_utc.dt.month>=4)&(raw.datetime_utc.dt.month<=9)]).best.rename("daylight time"),
    find_best_cor(cems[(cems.datetime_utc.dt.month>=11)|(cems.datetime_utc.dt.month<=2)],
        raw[(raw.datetime_utc.dt.month>=11)|(raw.datetime_utc.dt.month<=2)]).best.rename("standard time")],
    axis='columns')

cems_930_cors.to_csv("../../data/outputs/2021/cems_RAWeia930_cor_lags.csv")
cems_930_cors

In [ ]:
## Calculate correlations using different subsets of 930 data 

cems_930_cors = pd.concat([find_best_cor(cems, rolled_930).best.rename("all_years"),\
    find_best_cor(cems[cems.datetime_utc.dt.year==2019],rolled_930[rolled_930.datetime_utc.dt.year==2019]).best.rename("2019"),
    find_best_cor(cems[cems.datetime_utc.dt.year==2020],rolled_930[rolled_930.datetime_utc.dt.year==2020]).best.rename("2020"),
    find_best_cor(cems[cems.datetime_utc.dt.year==2021],rolled_930[rolled_930.datetime_utc.dt.year==2021]).best.rename("2021"),
    find_best_cor(cems[(cems.datetime_utc.dt.month>=4)&(cems.datetime_utc.dt.month<=9)],
        rolled_930[(rolled_930.datetime_utc.dt.month>=4)&(rolled_930.datetime_utc.dt.month<=9)]).best.rename("daylight time"),
    find_best_cor(cems[(cems.datetime_utc.dt.month>=11)|(cems.datetime_utc.dt.month<=2)],
        rolled_930[(rolled_930.datetime_utc.dt.month>=11)|(rolled_930.datetime_utc.dt.month<=2)]).best.rename("standard time")],
    axis='columns')

cems_930_cors.to_csv("../../data/outputs/2021/cems_RAWeia930_cor_lags.csv")
cems_930_cors

In [ ]:
# Visualize a BA for manual inspection

ba = "SC"

to_plot_930 = shifted[shifted.BA==ba].groupby("datetime_utc").sum()

print(f"correlations for {ba}")
print(cems_930_cors.loc[ba])

fig = go.Figure()
fig.add_trace(go.Scatter(x=cems[cems.ba_code==ba].datetime_utc, y=cems[cems.ba_code==ba].net_generation_mwh, name="CEMS"))
fig.add_trace(go.Scatter(x=to_plot_930.index, y=to_plot_930.generation, name="EIA 930 (after adjustment and rolling cleaning)"))
fig.update_layout(
    title=ba,
    xaxis_title="Date",
    yaxis_title="Generation"
)

# Interchange

In [ ]:
#interchange = pd.read_csv("../data/eia930/chalendar/EBA_rolling.csv",index_col=0, parse_dates=True)
interchanges = []
for year in [2019, 2020, 2021]: 
    interchange = pd.read_csv(f"../../data/outputs/{year}/eia930/eia930_raw.csv",index_col=0, parse_dates=True)
    interchange = interchange[interchange.index.year == year] # limit to after gen was reported by fuel type
    interchanges.append(interchange)

In [ ]:
interchange = pd.concat(interchanges)

In [ ]:
bas930 = {re.split(r"[-.]",c)[1] for c in interchange.columns}

In [ ]:
# given a df where columns are interchange data, add best correlation between matching BAs to interchange_cors dict
# optionally, write markdown to {file}.md and csvs at {file}_{ba}.csv
def interchange_cor(interchange, interchange_cors:dict={}, file="", name:str="cors"):
    # Delete file
    if file != "":
        hs = open(file+".md","w")
        hs.write("\n\n")
        hs.close() 

    for ba in bas930:
        print(ba, end="...")
        other_cols = [c for c in interchange.columns \
            if re.split(r"[-.]",c)[1]==ba \
                and re.split(r"[-.]",c)[2]!="ALL"]
        other_bas = [re.split(r"[-.]",c)[2] for c in other_cols]
        #print(f"{ba} connects to {other_bas}")

        out = pd.DataFrame(index=other_bas, columns=range(-12,12), dtype=float)
        for o_ba in out.index:
            this_way = f"EBA.{o_ba}-{ba}.ID.H"
            other_way = f"EBA.{ba}-{o_ba}.ID.H"
            if other_way not in interchange.columns or this_way not in interchange.columns: 
                continue
            for lag in out.columns:
                out.loc[o_ba,lag] = abs(interchange[this_way]\
                    .corr(-1*interchange[other_way].shift(lag)))
        
        # where is correlation the best?
        out = pd.concat([out, out.apply(lambda s: s.index[s.argmax()], axis=1).rename("best")], axis='columns')

        if file != "":
            # add new lines for proper markdown syntax
            hs = open(file+".md","a")
            hs.write(f"\n\n# {ba}\n\n")
            hs.close() 

            out.to_markdown(file+".md",mode="a")

            out.to_csv(f"{file}_{ba}"+".csv")

        interchange_cors[ba] = pd.concat([interchange_cors.get(ba, pd.DataFrame()), out.best.rename(name)], axis='columns')

    return interchange_cors


In [ ]:
int_cors = interchange_cor(interchange, interchange_cors={}, name="all_years")
int_cors = interchange_cor(interchange["2019-01-01T00:00":"2019-12-30T00:00"], int_cors, name="2019")
int_cors = interchange_cor(interchange["2020-01-01T00:00":"2020-12-30T00:00"], int_cors, name="2020")
int_cors = interchange_cor(interchange["2021-01-01T00:00":"2021-12-30T00:00"], int_cors, name="2021")
int_cors = interchange_cor(interchange[(interchange.index.month >= 4)&(interchange.index.month <=9)], int_cors, name="daylight savings")
int_cors = interchange_cor(interchange[(interchange.index.month >= 11)|(interchange.index.month <=2)], int_cors, name="standard time")


In [ ]:
# Inspect interchange correlations

int_cors["PJM"]

In [ ]:
# Output to md file because that's an easy way to manually scan through BAs and look for anomalies

file = "../../data/outputs/2021/interchange_corr_summary_adjusted.md"
hs = open(file,"w")
hs.write("\n\n")
hs.close() 

for (ba,out) in int_cors.items():

    # add new lines for proper markdown syntax
            hs = open(file,"a")
            hs.write(f"\n\n# {ba}\n\n")
            hs.close() 

            out.to_markdown(file,mode="a")

# Plot interchange for BA of interest

In [ ]:
ba1 = "AEC"
ba2 = "MISO"

fig = px.line(interchange[f"EBA.{ba1}-{ba2}.ID.H"])
fig.add_trace(go.Scatter(x=interchange.index, y=interchange[f"EBA.{ba2}-{ba1}.ID.H"], name=f"EBA.{ba2}-{ba1}.ID.H"))

In [ ]:
ba = "PJM"

# find cols of mappings in both directions 
other_cols = [c for c in interchange.columns \
    if re.split(r"[-.]",c)[1]==ba \
        and re.split(r"[-.]",c)[2]!="ALL"]
other_bas = [re.split(r"[-.]",c)[2] for c in other_cols]

these_cols = [f"EBA.{o_ba}-{ba}.ID.H" for o_ba in other_bas]

# make long version with just cols of interest, adding BA column and to/from column
toplot = pd.DataFrame()
for i in range(len(other_bas)): 
    to_add = (interchange[other_cols[i]]).rename("interchange").to_frame()
    to_add["source"] = ba
    to_add["BA"] = other_bas[i]

    to_add_2 = (interchange[these_cols[i]]*(-1)).rename("interchange").to_frame()
    to_add_2["source"] = "other BA"
    to_add_2["BA"] = other_bas[i]

    toplot = pd.concat([toplot, to_add, to_add_2], axis='index')


In [ ]:
fig = px.line(toplot, x=toplot.index, y="interchange", facet_col="BA", facet_col_wrap=2, color="source")
fig.update_layout(
    title=f"Interchange from {ba}",
    xaxis_title="Date",
    yaxis_title="Interchange",
    legend_title="Source for<br>interchange data"
)
fig.for_each_annotation(lambda a: a.update(text="Other "+a.text))

In [ ]:
first="PJM"
second="MISO"

fig = px.line(interchange, x=interchange.index, y=[f"EBA.{first}-{second}.ID.H",f"EBA.{second}-{first}.ID.H", f"EBA.{first}-ALL.TI.H"])

fig.update_layout(
    title=f"{first}/{second} interchange",
    xaxis_title="Date",
    yaxis_title="Interchange",
    legend_title="Series"
)

In [ ]:
ba = "CFE"

fig = go.Figure()
fig.add_trace(go.Scatter(x=interchange.index, 
    y=interchange[f"EBA.{ba}-ALL.D.H"]-interchange[f"EBA.{ba}-ALL.NG.H"]))

fig.update_layout(
    title=f"{ba} demand - generation",
    xaxis_title="Date",
    yaxis_title="Demand - generation",
    legend_title="Series"
)

# Sign issues across interchange data

Most interchanges should be negatively correlated with the interchange coming the other way. 

In [ ]:
# given a df where columns are interchange data, add best correlation between matching BAs to interchange_cors dict
# optionally, write markdown to {file}.md and csvs at {file}_{ba}.csv
def interchange_sign(interchange, i_sign:dict={}, file="", name:str="cors"):
    for ba in bas930:
        print(ba, end="...")
        other_cols = [c for c in interchange.columns \
            if re.split(r"[-.]",c)[1]==ba \
                and re.split(r"[-.]",c)[2]!="ALL"]
        other_bas = [re.split(r"[-.]",c)[2] for c in other_cols]
        #print(f"{ba} connects to {other_bas}")

        out = pd.DataFrame(index=other_bas, columns=range(-12,12), dtype=float)
        for o_ba in out.index:
            this_way = f"EBA.{o_ba}-{ba}.ID.H"
            other_way = f"EBA.{ba}-{o_ba}.ID.H"
            if other_way not in interchange or this_way not in interchange: 
                continue 
            for lag in out.columns:
                out.loc[o_ba,lag] = interchange[this_way]\
                    .corr(-1*interchange[other_way].shift(lag))
        
        # where is correlation the best?
        out = out.apply(lambda s: s.iloc[abs(s).argmax()], axis=1)

        i_sign[ba] = pd.concat([i_sign.get(ba, pd.DataFrame()), out.rename(name)], axis='columns')

    return i_sign


In [ ]:
int_sign = interchange_sign(interchange, {}, name="all_years")
int_sign = interchange_sign(interchange["2019-01-01T00:00":"2019-12-30T00:00"], int_sign, name="2019")
int_sign = interchange_sign(interchange["2020-01-01T00:00":"2020-12-30T00:00"], int_sign, name="2020")
int_sign = interchange_sign(interchange["2020-01-01T00:00":"2020-12-30T00:00"], int_sign, name="2021")
int_sign = interchange_sign(interchange[(interchange.index.month >= 4)&(interchange.index.month <=9)], int_sign, name="daylight savings")
int_sign = interchange_sign(interchange[(interchange.index.month >= 11)|(interchange.index.month <=2)], int_sign, name="standard time")

In [ ]:
file = f"{outputs_folder('2021')}/interchange_cors_sign.md"
hs = open(file,"w")
hs.write("\n\n")
hs.close() 

for (ba,out) in int_sign.items():

    # add new lines for proper markdown syntax
            hs = open(file,"a")
            hs.write(f"\n\n# {ba}\n\n")
            hs.close() 

            out.to_markdown(file,mode="a")